In [1]:
include("Code/envsetup.jl")
include("Code/constants.jl")
include("Code/functions.jl")
addprocs(cfg["nworkers"]; exeflags="--project=" * projectpath) # Call workers with this environment

println(stderr, "Using the configuration:")
println(stderr, cfg)
# # Jump operators and their derivatives
@everywhere using BackAction

  Activating project at `~/Documents/Research/MonitoringMetrology/notebooks`
   Resolving package versions...
  No Changes to `~/Documents/Research/MonitoringMetrology/notebooks/Project.toml`
  No Changes to `~/Documents/Research/MonitoringMetrology/notebooks/Manifest.toml`
   Resolving package versions...
  No Changes to `~/Documents/Research/MonitoringMetrology/notebooks/Project.toml`
  No Changes to `~/Documents/Research/MonitoringMetrology/notebooks/Manifest.toml`
  No Changes to `~/Documents/Research/MonitoringMetrology/notebooks/Project.toml`
  No Changes to `~/Documents/Research/MonitoringMetrology/notebooks/Manifest.toml`
Using the configuration:
Dict{String, Any}("GAMMA" => 0.5, "delta" => 1.0, "NCHANNELS0" => 1, "ntraj" => 100, "alphamin" => 0.0, "alphamax" => 100.5, "psi0e_re" => 1.0, "psi0g_re" => 0.0, "DELTA" => 0.0, "psi0e_im" => 0.0, "tffactor" => 31.0, "nworkers" => 4, "OMEGA" => 1.0, "ndeltas" => 50, "NLEVELS" => 2, "ntimes" => 200, "seed" => 1, "psi0g_im" => 0.0, "nal

In [2]:
using ForwardDiff, Statistics
import BackAction.MonitoringOperator as mo
using JLD2

In [3]:
const pauli_basis = [I(2), util.sigma_x, util.sigma_y, util.sigma_z]
const cfield = 0.0 + 0.0im
const alpha = [cfield]

1-element Vector{ComplexF64}:
 0.0 + 0.0im

In [4]:
directoryname = "Sample_$(real(cfield))+i$(imag(cfield))"
try
    mkdir(directoryname)
catch
    @warn "Directory already existed"
end 
    
const deltarange = collect(LinRange(0.0, 1.0, cfg["ndeltas"]))

metadata = Dict(
    "NCHANNELS0"      => NCHANNELS0,
    "NLEVELS"         => NLEVELS,
    "OMEGA"           => OMEGA,
    "GAMMA"           => GAMMA,
    "tf"              => tf,
    "tspan"           => string(tspan),
    "ntimes"          => cfg["ntimes"],
    "ntraj"           => cfg["ntraj"],
    "nworkers"        => cfg["nworkers"],
    "nchannels"       => nchannels,
    "nops"            => nops,
    "psi0"            => string(psi0),
    "u0"              => string(u0),
    "cfield"          => string(cfield),
    "tlist_first"     => first(tlist),
    "tlist_last"      => last(tlist),
)
# Write to TOML file
open(directoryname*"/metadata_$(real(cfield))+i$(imag(cfield)).toml", "w") do io
    TOML.print(io, metadata)
end


┌ Warning: Directory already existed
└ @ Main In[4]:5


# Calculating the Derivative fo the exponential
The idea is to use the following identity:
$$ \exp(-i\tau H_e) = e^{-|\alpha|^2\tau}e^{-ia_0(\theta)\tau}\left[ \cos(\tau\xi(\theta))-i\frac{\vec{a}(\theta)\cdot\vec{\sigma}}{\xi(\theta)}\sin(\tau\xi(\theta)) \right]$$

where 

$$ H(\theta) -i\alpha^* L(\theta) -i\frac{1}{2}L^\dagger L(\theta) = a_0(\theta) +\vec{a}(\theta)\cdot\vec{\sigma} $$

In [5]:
function get_a(theta)
    Htilde =H0(theta) - 1.0im*conj(cfield)*L0(theta)-0.5im*adjoint(L0(theta))*L0(theta) 
   a = [tr(pauli_basis[k]*Htilde)/tr(pauli_basis[k]^2) for k in 1:4] 
   return a[1], a[2], a[3], a[4]
end 


function get_Heffexponential(theta, tau)
    a0, a1, a2, a3 = get_a(theta)
    xi = sqrt(a1^2 + a2^2 + a3^2)
    dotproduct = a1*util.sigma_x + a2*util.sigma_y + a3*util.sigma_z
    return exp(-1.0im*a0*tau)*(cos(tau*xi)*I(2)-1.0im*dotproduct/xi *sin(tau*xi)) #the alpha factor may be ommited by normalization
end 

function get_Heffexponential!(cache, theta, tau)
    a0, a1, a2, a3 = get_a(theta)
    xi = sqrt(a1^2 + a2^2 + a3^2)
    cache .= exp(-1.0im*a0*tau)*(cos(tau*xi)*I(2)-1.0im*(a1*util.sigma_x +
                                 a2*util.sigma_y + a3*util.sigma_z)/xi *sin(tau*xi)) #the alpha factor may be ommited by normalization
end 


get_Heffexponential! (generic function with 1 method)

In [6]:
function monitoringstep!(cache_exp, cache_dexp, L, dL, cache_state, psi, cache_aux1, cache_aux2, cache_phi)
    mul!(psi, cache_exp, cache_state)  # This is exp(-i\tau H_e)\psi_n
    ####### PHI UPDATE without rescaling
    # Obtain  \partial_\theta exp(-i\tau*H_eff(\theta))*\psi , store in aux1
    mul!(cache_aux1, cache_dexp, cache_state) 
    # Obtain \partial_\theta exp(-i\tau*H_eff(\theta))*\psi + exp(-i\tau*H_eff(theta))*\phi, store where the derivative was
    mul!(cache_aux1, cache_exp, cache_phi, 1.0, 1.0)
    # Multiply by the jump operator and store in phi_cache
    mul!(cache_phi, L, cache_aux1)
    # Prepare the last term
    mul!(cache_aux2, dL, psi)
    # Now put everything together and store in cache_phi
    cache_phi .+= cache_aux2
    ###### STATE UPDATE without normalization
    mul!(cache_state, L, psi)
    ##### NORMALIZATION
    # Normalize phi
    normalization = norm(cache_state)
    lmul!(1 / normalization, cache_phi)
    # Normalize the after jump state
    lmul!(1 / normalization, cache_state);
end

monitoringstep! (generic function with 1 method)

In [7]:
function finalmonitoringstep!(cache_exp, cache_dexp, cache_state, psi, cache_aux1, cache_aux2, cache_phi)
    mul!(psi, cache_exp, cache_state)
    mul!(cache_aux1, cache_dexp, cache_state)
    mul!(cache_aux2, cache_exp, cache_phi)
    cache_phi .= cache_aux1 + cache_aux2
    lmul!(1/norm(psi), cache_phi)
    normalize!(psi)
end 

finalmonitoringstep! (generic function with 1 method)

In [8]:
L0 = d -> sqrt(GAMMA) * util.sigma_m
H0 = d -> d * [[0, 0] [0, 1.0 + 0im]] + 0.5 * OMEGA * util.sigma_x

#9 (generic function with 1 method)

In [9]:

function psiphi_finaltime!(sol, L, dL, cache_exp,cache_aux1, cache_aux2, psi, cache_state, cache_phi, delta)
    jumptimes = sol.prob.kwargs[:callback].continuous_callbacks[1].affect!.jump_times
    njumps = sol.prob.kwargs[:callback].continuous_callbacks[1].affect!.jump_counter[] - 1
    if njumps == 0
        tau = tf 
        get_Heffexponential!(cache_exp, delta, tau)
        ForwardDiff.derivative!(cache_dexp, theta -> get_Heffexponential(theta, tau), delta) 
        finalmonitoringstep!(cache_exp, cache_dexp, cache_state, psi, cache_aux1, cache_aux2, cache_phi)   
    else 
        tau = jumptimes[1]
        get_Heffexponential!(cache_exp, delta, tau)
        ForwardDiff.derivative!(cache_dexp, theta -> get_Heffexponential(theta, tau), delta) 
        monitoringstep!(cache_exp, cache_dexp, L, dL, cache_state, psi, cache_aux1, cache_aux2, cache_phi)
        if njumps > 1
            for k in 2:njumps
                tau = jumptimes[k] - jumptimes[k-1]
                get_Heffexponential!(cache_exp, delta, tau)
                ForwardDiff.derivative!(cache_dexp, theta -> get_Heffexponential(theta, tau), delta) 
                monitoringstep!(cache_exp, cache_dexp, L, dL, cache_state, psi, cache_aux1, cache_aux2, cache_phi)
            end
        end 
        
        tau = tf - jumptimes[end]
        get_Heffexponential!(cache_exp, delta, tau)
        ForwardDiff.derivative!(cache_dexp, theta -> get_Heffexponential(theta, tau), delta) 
        finalmonitoringstep!(cache_exp, cache_dexp, cache_state, psi, cache_aux1, cache_aux2, cache_phi)
    end
end

psiphi_finaltime! (generic function with 1 method)

In [11]:
filenametrsample = directoryname*"/trsample_$(real(cfield))+i$(imag(cfield)).bin"
open(filename, "w+") do io
        # Extend file to required size (m*n*element_size)
        seek(io, cfg["ndeltas"] * cfg["ntraj"] * sizeof(Float64))
        write(io, UInt8(0))
end

trsample = open(filenametrsample, "r+") do io
        Mmap.mmap(io, Matrix{Float64}, (cfg["ntraj"], cfg["ndeltas"]))
end



100×50 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 

In [13]:
Ls_par, H_par, He_par =  mo.obtain_parametric_unraveling_operators(L0, H0, T, [cfield], NLEVELS);
cache_aux1 = Vector{ComplexF64}(undef, NLEVELS)
cache_aux2 = Vector{ComplexF64}(undef, NLEVELS)
cache_state = copy(psi0)
psi = Vector{ComplexF64}(undef, NLEVELS)
cache_phi = zeros(ComplexF64, NLEVELS)
# Set the necessary operators
cache_exp = Matrix{ComplexF64}(undef, NLEVELS, NLEVELS)
cache_dexp = Matrix{ComplexF64}(undef, NLEVELS, NLEVELS)
sims = Dict()
for i in 1:cfg["ndeltas"]
    delta = deltarange[i]
    L = Ls_par[1](delta)
    dL = ForwardDiff.derivative(Ls_par[1], delta) 
   
    sim = obtain_ensol(L0, H0, delta, T, [cfield], params, tspan, e_ops, tlist);
    for k in 1:cfg["ntraj"]
            psiphi_finaltime!(sim[k], L, dL, cache_exp, cache_aux1, cache_aux2, psi, cache_state, cache_phi, delta )
            trsample[k, i] = 2*real(dot(psi, cache_phi))
            copyto!(cache_state, psi0)
            cache_phi .= 0.0 + 0.0im
    end 
    @save directoryname*"/sim_$(delta).jld2" sim
end


In [12]:
Mmap.sync!(trsample)
finalize(trsample)
